In [1]:
import json
from pprint import pprint
from matplotlib import pyplot as plt
import cv2
import copy
import operator
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, ZeroPadding2D, Dropout
from keras.callbacks import TensorBoard
from keras.models import load_model
from keras.optimizers import Adam
from time import time
import os
%matplotlib qt

Using TensorFlow backend.


In [2]:
# Read Individual JSON file
def readJson(file):
    with open(file, 'r') as fp:
        data = json.load(fp)
        
    return data

In [3]:
# Read JSON file names in a specified folder
def read_jsons(folder):
    jsons = [json for json in os.listdir(folder) if json.endswith(".json")]
    
    return jsons

In [4]:
# Read skeleton points for JSON files
def data_points(jsons, json_folder_dir):
    data_points = []
    
    for i in range(0, len(jsons)):
        data_points.append([])
        with open((json_folder_dir + jsons[i])) as f:
            data = json.load(f)
        
        n_people = len(data['people'])
    
        for j in range(0, n_people):
            data_points[i].append([])
            datas = data['people'][j]['pose_keypoints_2d']
        
            for k in range(0, len(datas)):
                data_points[i][j].append(datas[k])
                
    return data_points

In [5]:
# Classify what feature points represent which part of the body
def data_classification(data):
    body_label = {0:"Nose", 1:"Neck", 2:"RShoulder", 3:"RElbow", 4:"RWrist", 5:"LShoulder", 6:"LElbow", 7:"LWrist", 8:"MidHip", 9:"RHip", 10:"RKnee", 11:"RAnkle", 12:"LHip", 13:"LKnee",14:"LAnkle", 15:"REye", 16:"LEye", 17:"REar", 18:"LEar", 19:"LBigToe", 20:"LSmallToe", 21:"LHeel", 22:"RBigToe", 23:"RSmallToe", 24:"RHeel"}
    parts = {}
    people = {}
    
    for person in range(0, len(data)):
        points = data[person]
        for i in range(0, len(points)):
            parts[body_label[i]] = points[i]
        
        people[person] = parts
        parts = {}
        
    return people

In [6]:
# Filter feature points from all JSON files
def organise_data(data, number):
    total = len(data)
    people_body_point = {}
    
    for i in range(0, total):
        people = len(data[i])
        frame_points = {}
        id_number = 0
        if people != 0:
            for j in range(0, people):
                points = data[i][j]
                n_points = len(points)
                body_points = []
                data_certainty = 0
                for k in range(0, n_points - 1, 3):
                    body_points.append((points[0 + k], points[1 + k]))
                    data_certainty += points[2 + k]
                    
                
                data_certainty = data_certainty / number
                
                if data_certainty >= 0.5:
                    frame_points[id_number] = body_points
                    id_number += 1
                
            organised = data_classification(frame_points)
            people_body_point[i] = organised
                
    return people_body_point

In [7]:
# When there is only one person in a video, remove id on a person
def reformat(data):
    
    exercise = {}
    
    for i in data:
        for j in data[i]:
            exercise[i] = data[i][j]
            
    return exercise

In [8]:
# When two people are exercising, separate data sets into two
def section_separator(data, start, end):
    
    exercise_one = {}
    exercise_two = {}
    
                
    for i in range(start, end + 1):
        
        for j in data[str(i)]:
            if j == '0':
                exercise_one[i] = data[str(i)][j]
            else:
                exercise_two[i] = data[str(i)][j]
        
#         i += 1

    return exercise_one, exercise_two

In [9]:
# Regroup data from files to body parts
def organiseAllPoints(data):
    new_data = {}
    
    keys = ['Nose', 'Neck', 'RShoulder', 'RElbow', 'RWrist', 'LShoulder', 'LElbow', 'LWrist', 'MidHip', 'RHip', 'RKnee', 'RAnkle', 'LHip', 'LKnee', 'LAnkle', 'REye', 'LEye', 'REar', 'LEar', 'LBigToe', 'LSmallToe', 'LHeel', 'RBigToe', 'RSmallToe', 'RHeel']
    
    # Initialisation
    for i in range(len(keys)):
        
        new_data[keys[i]] = []
       
    for keys in data:
        for Lkeys in data[keys]:
            coordinate = new_data[Lkeys]
            
            points = data[keys][Lkeys]
            
            if points[0] != 0 and points[1] != 0:
                previousPoints = copy.deepcopy(points)
                coordinate.append(points)
            else:
                coordinate.append(previousPoints)
            
#             coordinate.append(points)
            
            new_data[Lkeys] = coordinate            
    
    return new_data

In [10]:
# Standardise data for individual file
def rescale(data):
    
    allPoints = organiseAllPoints(data)
        
    new_data = {}
    
    for keys in allPoints:
        
        new_data[keys] = []
        
    fit_points = []
    first = True
    
    for keys in allPoints:
        if first:
            fit_points = allPoints[keys]
            first = False
        else:
            fit_points + allPoints[keys]
            
    scaler = StandardScaler().fit(fit_points)
    
    for keys in allPoints:
        points = scaler.transform(allPoints[keys])
        new_data[keys] = points
        
    return new_data

In [11]:
# Merge two data sets into one data set
def merge(master, slave):
    original = copy.deepcopy(master)
    for i in master:
        masterList = master[i]
        slaveList = slave[i]
        
        finalList = np.concatenate((masterList, slaveList), axis = 0)
            
        original[i] = finalList
        
    return original

In [1]:
# Return division number depends on a file and an exercise
def returnNumber(superFolder, subFolder):
    number = 25
    
    if superFolder == 'Star_Jumps':
        if subFolder == 'Starjump10':
            number = 26
        if subFolder == 'Starjump7' or 'Starjump8' or 'Starjump9':
            number = 20
        
    if superFolder == 'Squats':
        if subFolder == 'Squats_21' or subFolder == 'Squats_22':
            number = 13
            
        if subFolder == 'Squats_23':
            number = 10
            
    if superFolder == 'Biceps':
        if subFolder == 'Bicep8' or subFolder == 'Bicep9' or subFolder == 'Bicep10':
            number = 20
            
        if subFolder == 'Bicep12':
            number = 13

        if subFolder == 'Bicep13' or subFolder == 'Bicep15':
            number = 16
        
        if subFolder == 'Bicep14':
             number = 18
            
        
    if superFolder == 'Chests':
        if subFolder == 'Chest6' or subFolder == 'Chest7' or subFolder == 'Chest8':
            number = 20
            
        if subFolder == 'Chest11' or subFolder == 'Chest12':
            number = 15

        if subFolder == 'Chest13':
            number = 14
            
    if superFolder == 'Punches':
        number = 20
        
        if subFolder == 'Punches12':
            number = 14
            
        if subFolder == 'Punches7' or subFolder == 'Punches8' or subFolder == 'Punches9':
            number = 11
        
    return number

In [13]:
# Read all the data from folders
def read_folders(superFolder):
    subFolders = next(os.walk('Sports_Skeleton/' + superFolder + '/.'))[1]
    
    keyStart = 0
    
    total = {}
    
    first = True
    
    for i in range(len(subFolders)):
        json_folder = 'Sports_Skeleton/' + superFolder + '/' + subFolders[i]
        
        number = returnNumber(superFolder, subFolders[i]) 
            
        json_folder_slash = json_folder + '/'
        
        jsons = read_jsons(json_folder)
        
        data_points1 = data_points(jsons, json_folder_slash)
        
        part_points = organise_data(data_points1, number)
                
        reformat_points = reformat(part_points)
        
        rescale_points = rescale(reformat_points)
        
        if first:
            total = rescale_points
            first = False
        else:
            for i in rescale_points:
                total = merge(total, rescale_points)
                                
    return total

In [14]:
# Convert from dictionary to list
def dicToList(data):
    bigList = []
    first = True
    for i in data:
        if first:
            bigList = data[i]
            first = False
        else:
             bigList = np.concatenate((bigList, data[i]), axis = 0)
    
    return bigList

In [15]:
# Find what is the maximum number people showed in a video. This is for debugging purposes
def findmaximumpeople(json_folder_dir, jsons, id0, idn):
    
    if not id0 == idn:
        midval = (id0 + idn)//2
        leftval = findmaximumpeople(json_folder_dir, jsons, id0, midval)
        rightval = findmaximumpeople(json_folder_dir, jsons, midval + 1, idn)
        
        if leftval >= rightval:
            return leftval
        else:
            return rightval
    else:
        with open((json_folder_dir + jsons[id0])) as f:
            data = json.load(f)
        
        number = 0
        
        for item in data['people']:
            data_points = item['pose_keypoints_2d']
            
            total_certainty = 0
            
            n_points = len(data_points)
            
            division = 0
            
            for k in range(0, n_points - 1, 3):
                total_certainty += data_points[2 + k]
                division += 1
                
#             total_certainty = total_certainty / 20
            total_certainty = total_certainty

        
            if (total_certainty >= 0.5):
                number += 1

                
        return number
    

# Reading all data sets

In [19]:
squats0 = read_folders('Squats')

In [21]:
arm0 = read_folders('Biceps')

In [22]:
starJump0 = read_folders('Star_Jumps')

In [23]:
chest0 = read_folders('Chests')

In [24]:
punch0 = read_folders('Punches')

In [26]:
ds2_data = readJson('Re-identified jsons/DS1.json')
ds3_data = readJson('Re-identified jsons/DS2.json')
ds4_data = readJson('Re-identified jsons/DS3.json')

In [27]:
squat1, arm1 = section_separator(ds2_data, 135, 800)
arm2, squat2 = section_separator(ds2_data, 1230, 1910)
squat3, arm3 = section_separator(ds2_data, 2700, 3350)
arm4, squat4 = section_separator(ds2_data, 3600, 4280)

In [28]:
squat5, arm5 = section_separator(ds3_data, 210, 930)
arm6, squat6 = section_separator(ds3_data, 1250, 1990)
squat7, arm7 = section_separator(ds3_data, 2330, 2980)
arm8, squat8 = section_separator(ds3_data, 3725, 4350)
squat9, arm9 = section_separator(ds3_data, 4585, 5325)

In [29]:
squat10, starJump1 = section_separator(ds4_data, 210, 980)
starJump2, squat11 = section_separator(ds4_data, 1430, 2210)
squat12, starJump3 = section_separator(ds4_data, 2550, 3330)
starJump4, squat13 = section_separator(ds4_data, 3580, 4370)

In [30]:
arms_rescale = {}
squats_rescale = {}
starJumps_rescale = {}

In [31]:
arms_rescale[0] = rescale(arm1)
arms_rescale[1] = rescale(arm2)
arms_rescale[2] = rescale(arm3)
arms_rescale[3] = rescale(arm4)
arms_rescale[4] = rescale(arm5)
arms_rescale[5] = rescale(arm6)
arms_rescale[6] = rescale(arm7)
arms_rescale[7] = rescale(arm8)
arms_rescale[8] = rescale(arm9)
squats_rescale[0] = rescale(squat1)
squats_rescale[1] = rescale(squat2)
squats_rescale[2] = rescale(squat3)
squats_rescale[3] = rescale(squat4)
squats_rescale[4] = rescale(squat5)
squats_rescale[5] = rescale(squat6)
squats_rescale[6] = rescale(squat7)
squats_rescale[7] = rescale(squat8)
squats_rescale[8] = rescale(squat9)
squats_rescale[9] = rescale(squat10)
squats_rescale[10] = rescale(squat11)
squats_rescale[11] = rescale(squat12)
squats_rescale[12] = rescale(squat13)
starJumps_rescale[0] = rescale(starJump1)
starJumps_rescale[1] = rescale(starJump2)
starJumps_rescale[2] = rescale(starJump3)
starJumps_rescale[3] = rescale(starJump4)

# Merging data sets

In [32]:
for i in range(0, 9):
    arm0 = merge(arm0, arms_rescale[i])

In [33]:
for i in range(0, 13):
    squats0 = merge(squats0, squats_rescale[i])

In [34]:
for i in range(0, 4):
    starJump0 = merge(starJump0, starJumps_rescale[i])

In [35]:
arm_size = len(arm0[list(arm0.keys())[0]])

In [36]:
squat_size = len(squats0[list(squats0.keys())[0]])

In [37]:
starJump_size = len(starJump0[list(starJump0.keys())[0]])

In [38]:
chest_size = len(chest0[list(chest0.keys())[0]])

In [39]:
punch_size = len(punch0[list(punch0.keys())[0]])

# Data sets conversion from dictionary to list

In [40]:
arms = dicToList(arm0)

In [41]:
squats = dicToList(squats0)

In [42]:
starJumps = dicToList(starJump0)

In [43]:
chests = dicToList(chest0)

In [44]:
punches = dicToList(punch0)

In [2]:
print(arms.shape)
print(squats.shape)
print(starJumps.shape)
print(chests.shape)
print(punches.shape)

NameError: name 'arms' is not defined

# Data sets reshaping

In [46]:
arms_shape = arms.reshape(arm_size, 50)
print(arms_shape.shape)

(978610, 50)


In [47]:
squats_shape = squats.reshape(squat_size, 50)
print(squats_shape.shape)

(873226, 50)


In [48]:
starJumps_shape = starJumps.reshape(starJump_size, 50)
print(starJumps_shape.shape)

(1093395, 50)


In [49]:
chests_shape = chests.reshape(chest_size, 50)
print(chests_shape.shape)

(740465, 50)


In [50]:
punches_shape = punches.reshape(punch_size,50)
print(punches_shape.shape)

(978296, 50)


# Cutting datasets for grouping

In [51]:
if len(arms_shape) % 30 != 0:
    arms_shape = arms_shape[:-(len(arms_shape) % 30), :]

if len(squats_shape) % 30 != 0:
    squats_shape = squats_shape[:-(len(squats_shape) % 30), :]
    
if len(starJumps_shape) % 30 != 0:
    starJumps_shape = starJumps_shape[:-(len(starJumps_shape) % 30), :]

if len(chests_shape) % 30 != 0:
    chests_shape = chests_shape[:-(len(chests_shape) % 30), :]

if len(punches_shape) % 30 != 0:
    punches_shape = punches_shape[:-(len(punches_shape) % 30), :]

print(arms_shape.shape)
print(squats_shape.shape)
print(starJumps_shape.shape)
print(chests_shape.shape)
print(punches_shape.shape)

(978600, 50)
(873210, 50)
(1093380, 50)
(740460, 50)
(978270, 50)


# Grouping

In [52]:
final_arm = arms_shape.reshape(int(len(arms_shape) / 30), 30, 50, 1)
final_arm_training, final_arm_test =  train_test_split(final_arm, test_size = 0.3)

print(final_arm_training.shape)

(22834, 30, 50, 1)


In [53]:
final_squats = squats_shape.reshape(int(len(squats_shape)/30), 30, 50, 1)
final_squats_training, final_squats_test = train_test_split(final_squats, test_size = 0.3)

print(final_squats_training.shape)

(20374, 30, 50, 1)


In [54]:
final_starJumps = starJumps_shape.reshape(int(len(starJumps_shape)/30), 30, 50, 1)
final_starJumps_training, final_starJumps_test = train_test_split(final_starJumps, test_size = 0.3)
print(final_starJumps_training.shape)

(25512, 30, 50, 1)


In [55]:
final_chests = chests_shape.reshape(int(len(chests_shape)/30), 30, 50, 1)
final_chests_training, final_chests_test = train_test_split(final_chests, test_size = 0.3)
print(final_chests_training.shape) 

(17277, 30, 50, 1)


In [56]:
final_punches = punches_shape.reshape(int(len(punches_shape)/30), 30, 50, 1)
final_punches_training, final_punches_test = train_test_split(final_punches, test_size = 0.3)
print(final_punches_training.shape)

(22826, 30, 50, 1)


# Labels for training data sets and test data sets

In [57]:
label = np.array([])

for i in range(int(len(final_arm_training))):
    label = np.append(label, 0)

for i in range(int(len(final_squats_training))):
    label = np.append(label, 1)
    
for i in range(int(len(final_starJumps_training))):
    label = np.append(label, 2)
    
for i in range(int(len(final_chests_training))):
    label = np.append(label, 3)

for i in range(int(len(final_punches_training))):
    label = np.append(label, 4)
    
print(label.shape)
    

(108823,)


In [80]:
dissertation_label = np.array([])

for i in range(int(len(final_arm_test))):
    dissertation_label = np.append(dissertation_label, 0)
    
for i in range(int(len(final_squats_test))):
    dissertation_label = np.append(dissertation_label, 1)
    
for i in range(int(len(final_starJumps_test))):
    dissertation_label = np.append(dissertation_label, 2)
    
for i in range(int(len(final_chests_test))):
    dissertation_label = np.append(dissertation_label, 3)
    
for i in range(int(len(final_punches_test))):
    dissertation_label = np.append(dissertation_label, 4)

In [59]:
x_data = np.concatenate((final_arm_training, final_squats_training, final_starJumps_training, final_chests_training, final_punches_training))

In [60]:
x_data.shape

(108823, 30, 50, 1)

In [61]:
label = label.reshape((int(len(final_arm_training))) + (int(len(final_squats_training))) + (int(len(final_starJumps_training))) + (int(len(final_chests_training))) + (int(len(final_punches_training))), 1)

In [62]:
label.shape

(108823, 1)

In [63]:
dissertation_test = np.concatenate((final_arm_test, final_squats_test, final_starJumps_test, final_chests_test, final_punches_test))

In [81]:
dissertation_label = dissertation_label.reshape((int(len(final_arm_test))) + (int(len(final_squats_test))) + (int(len(final_starJumps_test))) + (int(len(final_chests_test))) + (int(len(final_punches_test))), 1)

In [75]:
dissertation_label.shape

(46641, 1)

In [66]:
dissertation_test.shape

(46641, 30, 50, 1)

In [82]:
y_train = to_categorical(label)
dissertation_label = to_categorical(dissertation_label)

In [69]:
x_data.shape

(108823, 30, 50, 1)

# Model Creation

In [70]:
model = Sequential()
model.add(Conv2D(64, kernel_size = 5, activation = 'relu', input_shape = (30, 50, 1)))

model.add(ZeroPadding2D((1,1)))
model.add(Dropout(0.1))
model.add(Conv2D(32, kernel_size = 4, activation = 'relu'))

model.add(ZeroPadding2D((1,1)))
model.add(Dropout(0.1))
model.add(Conv2D(16, kernel_size = 3, activation = 'relu'))

model.add(ZeroPadding2D((1,1)))
model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size = (8,8), strides = (3,3)))
model.add(Flatten())
model.add(Dense(5, activation='softmax'))

tensorboard = TensorBoard(log_dir='log/{}'.format(time()))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [71]:
model.compile(optimizer = Adam(lr = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Training

In [83]:
model.fit(x_data, y_train, validation_data =(dissertation_test, dissertation_label), epochs=100, callbacks=[tensorboard], verbose=False, shuffle=True)

Instructions for updating:
Use tf.cast instead.


In [84]:
model.save('model/Exercise_Classification_Shuffle_Fixed_Data_Dissertation.h5')

In [85]:
del model

# TEST

In [87]:
def exercise(i):
    switcher={
            0:'Biceps',
            1:'Squats',
            2:'Star_Jumps',
            3:'Chests',
            4:'Punches',
    }
    
    return switcher.get(i,"Invalid number")

In [88]:
def classify(result):
    score = np.zeros(5)
    
    for i in range(len(result)):
        index = np.argmax(result[i])
        
        score[index] += 1
    
    total = np.sum(score)
    
    percentage =np.around(score / total, decimals=2)
    
    order = percentage.argsort()[-4:][::-1]
    
    return [(exercise(order[0]), percentage[order[0]]) , (exercise(order[1]), percentage[order[1]]), (exercise(order[2]), percentage[order[2]]), (exercise(order[3]), percentage[order[3]])]    

In [86]:
model = load_model('model/Exercise_Classification_Shuffle_Fixed_Data_Test.h5')

# Arm

In [89]:
prediction = model.predict(final_arm_test)

In [92]:
exercise_prediction = classify(prediction)

In [93]:
print(exercise_prediction)

[('Biceps', 0.82), ('Chests', 0.16), ('Squats', 0.02), ('Punches', 0.0)]

# Squats

In [94]:
prediction = model.predict(final_squats_test)

In [96]:
print(classify(prediction))

[('Squats', 1.0), ('Punches', 0.0), ('Chests', 0.0), ('Star_Jumps', 0.0)]

# Starjump

In [97]:
prediction = model.predict(final_starJumps_test)

In [98]:
print(classify(prediction))

[('Star_Jumps', 0.99), ('Biceps', 0.01), ('Punches', 0.0), ('Chests', 0.0)]

# Chest

In [99]:
prediction = model.predict(final_chests_test)

In [100]:
print(classify(prediction))

[('Chests', 0.83), ('Biceps', 0.11), ('Squats', 0.05), ('Punches', 0.0)]

# Punch

In [101]:
prediction = model.predict(final_punches_test)

In [102]:
print(classify(prediction))

[('Punches', 0.81), ('Biceps', 0.1), ('Chests', 0.08), ('Squats', 0.01)]
